서울소방청out 파일 시각화

In [1]:
import numpy as np
import pandas as pd
import requests
from urllib.parse import quote
import folium, json

In [46]:
df1 = pd.read_csv('../daou/서울소방청out')
df1.head()

,Unnamed: 0,관서명,주소,번호
0,0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119


In [47]:
# 1 .. KAKAO API로 위경도값 추출
# api 로드
with open(r'C:\Users\tjoeun\Desktop\수업자료\수업_실습\API\카카오API.txt') as file:
    kakao_key = file.read()

In [48]:
# 주소 호출 및 삽입
search_url = 'https://dapi.kakao.com/v2/local/search/address.json'
lat_list, lon_list= [], []

for i in df1.index:    
    url = f'{search_url}?query={quote(df1.주소[i])}'
    result = requests.get(url, headers={"Authorization": f'KakaoAK {kakao_key}'}).json()
    #print(result)
    try:
        lon_list.append(float(result['documents'][0]['x']))
        lat_list.append(float(result['documents'][0]['y']))
    except:
        print(df1.관서명[i], df1.주소[i], " 의 위경도 정보를 찾을 수 없습니다")
        #df1.주소[i] = "서울 은평구 수색로 320"
        lon_list.append(np.nan)
        lat_list.append(np.nan)

df1['위도'] = lat_list
df1['경도'] = lon_list

은평소방서-수색-119 안전센터 서울특별시 은평구 수색로 294 (수색동)  의 위경도 정보를 찾을 수 없습니다


In [50]:
df1.info()
df1.head(31)
df1.drop(30,axis=0, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  100 non-null    int64  
 1   관서명         100 non-null    object 
 2   주소          100 non-null    object 
 3   번호          100 non-null    object 
 4   위도          99 non-null     float64
 5   경도          99 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 4.8+ KB


In [54]:
# .. 지도에 정보 표시
import folium

map = folium.Map(location=[df1.위도.mean(),df1.경도.mean()], zoom_start=12)

for i in df1.index:
    folium.Marker(location=[df1.위도[i],df1.경도[i]],
                  popup=folium.Popup(df1.주소[i], max_width=200),
                  tooltip=df1.관서명[i],
                  icon = folium.Icon(icon='glyphicon glyphicon-home')
                  ).add_to(map)
title = '<h3 align="center" style="font-size:20"> 서울시 소방소</h3>'
map.get_root().html.add_child(folium.Element(title))
map
